In [5]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import datetime
from urllib.request import urlopen
import requests
import time

In [6]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [7]:
# 다음 사이트로 진행
url = 'https://search.daum.net/search?w=tv&q=%EC%8A%88%ED%8D%BC%EB%B0%B4%EB%93%9C2%20%EC%8B%9C%EC%B2%AD%EB%A5%A0&irt=tv-program&irk=90634&DA=TVP'

driver.get(url)
time.sleep(3)

In [8]:
# 더보기 진행
for i in range(1):
    xpath = """//*[@id="tv_rating"]/div[3]/div[2]/a"""
    driver.find_element_by_xpath(xpath).click()

In [9]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html class="win chrome" lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="슈퍼밴드2 시청률 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=tv&amp;q=%EC%8A%88%ED%8D%BC%EB%B0%B4%EB%93%9C2%20%EC%8B%9C%EC%B2%AD%EB%A5%A0" property="og:url"/>
<meta content="Daum 검색에서 슈퍼밴드2 시청률에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/og_search.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<title>슈퍼밴드2 시청률 – Daum 검색</title>
<link href="//search.daum.net/OpenSearch.xml" rel="search" title="Daum" type="application/opensearchdescription+xml"/>
<link charset="utf-8"

In [10]:
viewr_ratings = [] # 시청률 넣을 리스트
xpath_basic = """//*[@id="tv_rating"]/div[3]/table/tbody/tr[""" # xpath 기본

for i in np.arange(1, 15):
    tmp = str(i) + "]/td[3]" # 각 회차에 대한 시청률 얻어오기 위함
    xpath = xpath_basic + tmp # 시청률을 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(xpath)
    view = find_element.get_attribute('innerHTML')
    viewr_ratings.append(view)

In [11]:
# 14회, 13회, 12회, ... , 2회, 1회 순서
viewr_ratings

['3.861%',
 '3.08%',
 '1.936%',
 '3.279%',
 '4.178%',
 '3.672%',
 '3.957%',
 '3.74%',
 '3.68%',
 '3.592%',
 '4.047%',
 '3.273%',
 '3.542%',
 '3.081%']

In [52]:
# 회차, 시청률 저장할 데이터프레임
superband_viewr = pd.DataFrame(columns=['회차', '시청률', '회차_merge용'])
superband_viewr

,회차,시청률,회차_merge용


In [53]:
# 데이터프레임에 저장
for i in np.arange(0, len(viewr_ratings)):
    episode = 14 - i # 회차 계산
    merge_col = "슈퍼밴드2 " + str(episode) + "회"
    dic = {"회차" : episode,
           "시청률" : viewr_ratings[i],
           "회차_merge용" : merge_col}
    
    superband_viewr = superband_viewr.append(dic, ignore_index=True)

superband_viewr

,회차,시청률,회차_merge용
0,14,3.861%,슈퍼밴드2 14회
1,13,3.08%,슈퍼밴드2 13회
2,12,1.936%,슈퍼밴드2 12회
3,11,3.279%,슈퍼밴드2 11회
4,10,4.178%,슈퍼밴드2 10회
5,9,3.672%,슈퍼밴드2 9회
6,8,3.957%,슈퍼밴드2 8회
7,7,3.74%,슈퍼밴드2 7회
8,6,3.68%,슈퍼밴드2 6회
9,5,3.592%,슈퍼밴드2 5회


In [57]:
# 시청률 csv 파일로 저장
superband_viewr.to_csv('superband_viewr.csv', index=False, encoding="utf-8-sig")

In [54]:
# 네이버 tv 크롤링 결과 가져오기
superband_view = pd.read_csv('superband_view.csv')
superband_view.rename(columns = {"회차" : "회"}, inplace=True)
superband_view

,회,영상 이름,조회수,좋아요 수
0,슈퍼밴드2 14회,[파이널] TOP6 등장! 제 2대 슈퍼밴드의 주인공..,8887,26
1,슈퍼밴드2 14회,＂보여드리겠습니다＂ 개성 뚜렷한 TOP6의 뜨거운..,5079,25
2,슈퍼밴드2 14회,＂1등 못하면 억울할 것 같아요( •̀ ω •́ )✧＂ 승..,4282,66
3,슈퍼밴드2 14회,온몸이 들썩😆 희망 가득한 카디의 자작곡 〈Rea..,21812,566
4,슈퍼밴드2 14회,"🎉축하🎉 유일무이 국악 파이널리스트 박다울, ..",7096,92
...,...,...,...,...
455,슈퍼밴드2 1회,"피처링 러브콜 1순위✨ R&B 신예, 대세 보컬리스..",9106,25
456,슈퍼밴드2 1회,음색 깡패♨ 밴드 보컬로 도전하는 문수진의 〈Se..,34968,338
457,슈퍼밴드2 1회,[단독 선공개] 어느 쪽을 보고 노래를..💦 생애..,68448,461
458,슈퍼밴드2 1회,슈퍼밴드2 2회 예고편,20892,45


In [56]:
superband = pd.merge(superband_viewr, superband_view, left_on='회차_merge용', right_on='회')
superband = superband.drop(['회차_merge용', '회'], axis=1)
superband

,회차,시청률,영상 이름,조회수,좋아요 수
0,14,3.861%,[파이널] TOP6 등장! 제 2대 슈퍼밴드의 주인공..,8887,26
1,14,3.861%,＂보여드리겠습니다＂ 개성 뚜렷한 TOP6의 뜨거운..,5079,25
2,14,3.861%,＂1등 못하면 억울할 것 같아요( •̀ ω •́ )✧＂ 승..,4282,66
3,14,3.861%,온몸이 들썩😆 희망 가득한 카디의 자작곡 〈Rea..,21812,566
4,14,3.861%,"🎉축하🎉 유일무이 국악 파이널리스트 박다울, ..",7096,92
...,...,...,...,...,...
455,1,3.081%,"피처링 러브콜 1순위✨ R&B 신예, 대세 보컬리스..",9106,25
456,1,3.081%,음색 깡패♨ 밴드 보컬로 도전하는 문수진의 〈Se..,34968,338
457,1,3.081%,[단독 선공개] 어느 쪽을 보고 노래를..💦 생애..,68448,461
458,1,3.081%,슈퍼밴드2 2회 예고편,20892,45


In [ ]:
# 시청률 csv 파일로 저장
superband_viewr.to_csv('superband_viewr.csv', index=False, encoding="utf-8-sig")